In [1]:
import os
import pandas as pd
import ast

# Read the original CSV file
input_csv = 'player_market_values.csv'  # Replace with your file name
df = pd.read_csv(input_csv)

# Create a new folder to store the CSV files
output_folder = 'data'
os.makedirs(output_folder, exist_ok=True)

# Process each row in the DataFrame
for _, row in df.iterrows():
    player_name = row['Name']
    market_values = ast.literal_eval(row['Market Values (₹ Cr)'])  # Convert string list to a Python list
    
    # Create a new DataFrame for the player
    player_df = pd.DataFrame({
        'game': range(1, len(market_values) + 1),
        'market value': market_values
    })
    
    # Save the new CSV file in the 'data' folder
    output_file = os.path.join(output_folder, f"{player_name}.csv")
    player_df.to_csv(output_file, index=False)

print(f"CSV files created successfully in the '{output_folder}' folder!")


CSV files created successfully in the 'data' folder!


In [2]:
import os
import pandas as pd

# Set up input and output directories
input_folder = 'data'
output_folder = 'return'
os.makedirs(output_folder, exist_ok=True)

# Process each CSV file in the 'data' folder
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):  # Only process CSV files
        input_file = os.path.join(input_folder, filename)
        
        # Read the player's data CSV
        df = pd.read_csv(input_file)
        
        # Calculate returns
        df['return'] = df['market value'].pct_change()
        
        # Create the new DataFrame with game and return
        return_df = df[['game', 'return']].dropna()  # Drop NaN for the first game
        
        # Save the new CSV file in the 'return' folder
        output_file = os.path.join(output_folder, filename)
        return_df.to_csv(output_file, index=False)

print(f"Return CSV files created successfully in the '{output_folder}' folder!")


Return CSV files created successfully in the 'return' folder!


In [4]:
import os
import pandas as pd

# Input folder containing return CSV files
input_folder = 'return'
output_file = 'consolidated_returns.csv'

# Initialize an empty dictionary to store data
data = {'game': None}  # Start with the 'game' column

# Process each CSV file in the 'return' folder
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):  # Only process CSV files
        player_name = filename.replace('.csv', '')  # Extract player name from filename
        file_path = os.path.join(input_folder, filename)
        
        # Read the return CSV
        df = pd.read_csv(file_path)
        
        # Set the 'game' column as the index if not already done
        if data['game'] is None:
            data['game'] = df['game']
        
        # Add the player's return column
        data[player_name] = df['return']

# Create a DataFrame from the collected data
consolidated_df = pd.DataFrame(data)

# Save the consolidated CSV
consolidated_df.to_csv(output_file, index=False)

print(f"Consolidated CSV file created successfully: {output_file}")


Consolidated CSV file created successfully: consolidated_returns.csv


In [ ]:
import pandas as pd
import json

# Input and output file paths
input_csv = 'consolidated_returns.csv'
output_json = 'structured_returns.json' 

# Read the consolidated CSV file
df = pd.read_csv(input_csv)

# Initialize the list to hold the structured data
structured_data = []

# Iterate through each player column (excluding 'game')
for idx, player_name in enumerate(df.columns[1:], start=1):
    entry = {
        "id": idx,
        player_name: df[player_name].fillna(0).tolist()  # Fill NaN with 0 and convert to list
    }
    structured_data.append(entry)

# Save the structured data to JSON
with open(output_json, 'w') as f:
    json.dump(structured_data, f, indent=4)

print(f"JSON file created successfully: {output_json}")


JSON file created successfully: structured_returns.json


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
from cvxopt import matrix, solvers

# # Load the consolidated returns CSV
# df = pd.read_csv('consolidated_returns.csv')

# # Input array of column indices (e.g., 2, 3, 5, 6, 10)
# input_columns = [6, 8, 10, 14, 15]

# Select relevant columns (ignoring the 'game' column)
data_matrix = df.iloc[:, input_columns].to_numpy()

print(data_matrix)
# 1. Dimensions of the data matrix
print("Dimensions of data matrix:", data_matrix.shape)

# 2. Variance-covariance matrix
cov_matrix = np.cov(data_matrix, rowvar=False)
print("Variance-Covariance Matrix:\n", cov_matrix)
print("Dimensions of Covariance Matrix:", cov_matrix.shape)

# 3. Mean vector
mean_vector = np.mean(data_matrix, axis=0)
print("Mean Vector:\n", mean_vector)
print("Length of Mean Vector:", len(mean_vector))

# 4. Unbounded closed-form solution
one_vector = np.ones(len(mean_vector))

# Inverse of covariance matrix
inv_cov_matrix = np.linalg.inv(cov_matrix)

# Define constraints
num_assets = cov_matrix.shape[0]

# Dmat: Quadratic term (2 * covariance matrix)
Dmat = 2 * cov_matrix

# dvec: Linear term (set to 0 as there is no linear term in the objective function)
dvec = np.zeros(num_assets)

# Equality constraint: sum(weights) = 1
A = np.ones((1, num_assets))  # 1 x num_assets matrix for equality constraint
b = np.array([1.0])  # RHS for equality: sum(weights) = 1

# Inequality constraint: weights >= 0
G = -np.eye(num_assets)  # Negative identity matrix for weights >= 0
h = np.zeros(num_assets)  # RHS for weights >= 0

# Convert inputs to cvxopt format
Dmat = matrix(Dmat)
dvec = matrix(dvec)
G = matrix(G)
h = matrix(h)
A = matrix(A)
b = matrix(b)

# Solve quadratic programming problem
solution = solvers.qp(Dmat, dvec, G, h, A, b)  # A and b are equality constraints

# Extract the solution
weights = np.array(solution['x']).flatten()

# Print the optimal weights
print("Optimal Weights:", weights)

[[-0.125      -0.1        -0.15384615 -0.11111111  0.16666667]
 [-0.14285714 -0.05555556 -0.09090909  0.          0.28571429]
 [-0.16666667  0.         -0.2         0.          0.11111111]
 [-0.2         0.         -0.1875      0.          0.2       ]
 [-0.25       -0.11764706 -0.15384615  0.          0.33333333]
 [-0.33333333  0.         -0.30909091  0.          0.25      ]
 [-0.2        -0.13333333 -0.36842105  0.125       0.1       ]
 [-0.25       -0.23076923 -0.375       0.11111111  0.01818182]
 [-0.33333333 -0.2        -0.93333333  0.          0.25      ]]
Dimensions of data matrix: (9, 5)
Variance-Covariance Matrix:
 [[ 0.00575743  0.00183692  0.01334168 -0.00138682 -0.00167176]
 [ 0.00183692  0.00750771  0.0128722  -0.00244592  0.00253366]
 [ 0.01334168  0.0128722   0.06500278 -0.00401575  0.0014158 ]
 [-0.00138682 -0.00244592 -0.00401575  0.00482253 -0.00347723]
 [-0.00167176  0.00253366  0.0014158  -0.00347723  0.01017403]]
Dimensions of Covariance Matrix: (5, 5)
Mean Vector:


In [29]:
# weights = [3.01595859e-01, 3.70545065e-03, 2.03965186e-07, 4.28635222e-01, 2.66063265e-01]

# Now `numbers` is a Python list containing the values.
money_to_invest= 1_000_000

In [30]:
money_to_all=[]
for i in weights:
    money_to_all.append(money_to_invest*i)

In [ ]:
money_to_all

np.float64(258373.71561072487)